# Load modules

In [1]:
import sys
sys.path.append("./modules")
from datetime import datetime
from IPython.display import HTML
# Loading cleaning-merging and aggregating-transforming classes
from clean_merge import *
from aggregate_transform import *

# Set date to create dataset for

In [2]:
#dates = ["2018-01-01", "2017-01-01", "2016-01-01", "2015-01-01", "2014-01-01", "2013-01-01"]
#list_date_dataset = [datetime.strptime(date, '%Y-%m-%d') for date in dates]
date_dataset = datetime.strptime("2017-01-01", '%Y-%m-%d')

# Setup cleaner-merger and aggregator-transformer

In [3]:
project = 'graydon-moving-indicator'
bucket = 'graydon-data'
# Create object for cleaning and merging
clean_merge = Clean_Merge(name_project = project, name_bucket = bucket, 
                          dir_input_data = '01_input', dir_output_data = '02_cleaned')
# Create object for aggregating and transforming
aggregate_transform = Aggregate_Transform(name_project = project, name_bucket = bucket,
                                          dir_input_data = '02_cleaned', dir_output_data = '03_aggregated')

# Executing cleaning-merging, aggregating-transforming

In [ ]:
df_month_year = clean_merge.clean_merge_data(date_dataset)

Reading and cleaning features for 2017-01-01


In [ ]:
df_year = aggregate_transform.aggregate_transform_df(df = df_month_year, 
                                                     date_dataset)